In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import base64

#import crud class
from crud import AnimalShelter



###########################
# Data Manipulation / Model
###########################

#Instantiate AnimalShelter class with username and password
username = "aacuser"
password = "user"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog"}))





#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    
    
    html.Hr(),    
    html.H2(children='Michael Delfino :)'),
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        
             ]),
    html.Br(),
    html.Hr(),
    
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water', 'value':'W'},
            {'label': 'Mountain or Wilderness', 'value':'MW'},
            {'label': 'Disaster or Individual Tracking', 'value':'DIT'},
            {'label': 'Reset', 'value':'R'}
        ],
        value = 'R'
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        sort_action = "native", 
    ),    
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        
        html.Div(
            id='map-id',
            className='col s12 m6',
            ),
                 
        html.Div(
            dcc.Graph('graph-id')
            ),
                 
        ]),
    
])
#############################################
# Interaction Between Components / Controller
#############################################


    
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', 'figure'),
    [Input('datatable-id', 'derived_viewport_data')]
)
def update_graphs(viewData):    
    dff = pd.DataFrame.from_dict(viewData)
    
    #plot = dff.plot.pie(y=dff['breed'], figsize=(5,5))
    
    figure=px.pie(dff, values=dff['age_upon_outcome_in_weeks'], names=dff['breed'])
    return plot

@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_viewport_data')]
)
def update_map(viewData):    
    
    dff = pd.DataFrame.from_dict(viewData)
    
    return [       
        dl.Map(style={"width": '1000px', 'height': '500px'}, center=[dff.loc[0, 'location_lat'], dff.loc[0, 'location_long']], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),            
            #Marker with tool tip and popup
            dl.Marker(position=[dff.loc[0, 'location_lat'],dff.loc[0, 'location_long']], children =[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
              ])
            ])
            
       ])
    ]

@app.callback(
    [Output('datatable-id','data'), Output('datatable-id', 'columns')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):

        if (filter_type == 'W'):
            df = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog", "breed":{"$in":["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}, "sex_upon_outcome":"Intact Female", "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        if (filter_type == 'MW'): 
            df = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog", "breed":{"$in":["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]}, "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":26},"age_upon_outcome_in_weeks":{"$lte":156}}))
        if (filter_type == 'DIT'):
            df = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog", "breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]}, "sex_upon_outcome":"Intact Male", "age_upon_outcome_in_weeks":{"$gte":20},"age_upon_outcome_in_weeks":{"$lte":300}}))  
        if (filter_type == 'R'):
            df = pd.DataFrame.from_records(shelter.read({}))
            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)
    

app